In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter

# Load the dataset
data = pd.read_csv('dataset.csv')

# Assuming the dataset has features in columns 'pixel1', 'pixel2', ..., 'pixelN' and the target is labeled 'label'
X = data.drop('label', axis=1).values
y = data['label'].values

# Split the data into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the custom KNN Classifier
class CustomKNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        predictions = [self._predict(x) for x in X_test]
        return np.array(predictions)

    def _predict(self, x):
        # Compute distances between x and all examples in the training set
        distances = [np.linalg.norm(x - x_train) for x_train in self.X_train]
        
        # Get the indices of the k nearest neighbors
        k_indices = np.argsort(distances)[:self.k]
        
        # Get the labels of the k nearest neighbors
        k_neighbor_labels = [self.y_train[i] for i in k_indices]
        
        # Majority vote, most common class label
        most_common = Counter(k_neighbor_labels).most_common(1)
        return most_common[0][0]

# Initialize the KNN model
knn = CustomKNN(k=3)

# Train the model
knn.fit(X_train, y_train)

# Make predictions
y_pred = knn.predict(X_test)

# Evaluate the model
accuracy = np.sum(y_pred == y_test) / len(y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 96.06%


In [2]:
import joblib

# Save the trained KNN model
joblib.dump(knn, 'model/digit_recognizer_3')


['model/digit_recognizer_3']

In [ ]:
import joblib
import cv2
import numpy as np
import pyscreenshot as ImageGrab
import time

# Load the updated pre-trained SVM model
model = joblib.load("model/digit_recognizer_3")
image_folder = "./img/"

while True:
    # Capture the screen region
    img = ImageGrab.grab(bbox=(60, 470, 400, 870))
    img.save(image_folder + "img.png")
    
    # Preprocess the image
    im = cv2.imread(image_folder + "img.png")
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im_gray = cv2.GaussianBlur(im_gray, (15, 15), 0)
    
    # Threshold the image
    _, im_th = cv2.threshold(im_gray, 100, 255, cv2.THRESH_BINARY)
    roi = cv2.resize(im_th, (28, 28), interpolation=cv2.INTER_AREA)
    
    # Normalize the image
    roi = roi / 255.0
    X = roi.flatten()  # Flatten into a single array
    
    # Predict the digit
    predictions = model.predict([X])
    print("Prediction:", predictions[0])
    
    # Display the result
    cv2.putText(im, f"Prediction: {predictions[0]}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Result", im)
    
    # Wait for 10 seconds
    time.sleep(5)

    # Break on Enter key
    if cv2.waitKey(1) == 13:  # Press Enter to exit
        break

cv2.destroyAllWindows()


Prediction: 3
Prediction: 3
Prediction: 3
Prediction: 3
Prediction: 3
Prediction: 3
Prediction: 6
Prediction: 6
Prediction: 7
Prediction: 7
Prediction: 7
Prediction: 7
Prediction: 6
Prediction: 6
Prediction: 7
Prediction: 7
Prediction: 9
Prediction: 7
Prediction: 7
Prediction: 7
Prediction: 0
Prediction: 0
Prediction: 3
Prediction: 7
Prediction: 7
Prediction: 3
Prediction: 3
Prediction: 3
Prediction: 3
Prediction: 3
Prediction: 3
Prediction: 3
Prediction: 3
